In [101]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict, Literal
from pydantic import BaseModel, Field, ValidationError
from dotenv import load_dotenv
import json

In [102]:
#load environment variables from a .env file
load_dotenv()

#initiate the language model fir structured output
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [103]:
class SentimentSchema(BaseModel):
    sentiment: Literal["positive", "negative"] = Field(description="The sentiment of the review")

In [104]:
structured_model = model.with_structured_output(SentimentSchema)

In [105]:
# prompt = 'what the the sentiment of review - The software is not'
# response = structured_model.invoke(prompt)


In [106]:
class ReviewState(TypedDict):
    review: str
    sentiment: Literal["positive", "negative"]
    diagonosis: dict
    response: str

In [107]:
def find_sentiment(state: ReviewState):
    prompt = f'for the following review find out the sentiment - {state["review"]}'
    response = structured_model.invoke(prompt).sentiment
    return {"sentiment": response}


In [108]:
graph = StateGraph(ReviewState)
graph.add_node('find_sentiment',find_sentiment)

graph.add_edge(START,'find_sentiment')
graph.add_edge('find_sentiment',END)

workflow = graph.compile()

In [109]:
initial_state = {
    'review': 'The software is not user-friendly and crashes frequently'
}

final_state = workflow.invoke(initial_state)
print(final_state)

{'review': 'The software is not user-friendly and crashes frequently', 'sentiment': 'negative'}
